<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Intro_for_experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [112]:
#Load the data set 
mnist = tf.keras.datasets.mnist.load_data()

#Split the dataset 
(train_images,train_labels),(test_images,test_labels) = mnist

#Standardize/ normalize the data 
train_images,test_images = train_images/255.0,test_images/255.0 

#Add a channels dimension 
train_images = train_images[..., tf.newaxis].astype('float32')
test_images = test_images[...,tf.newaxis].astype('float32')

In [113]:
#Batch and shuffle the dataset 
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [114]:
#Build the model using the subclassing API
class MyModel(tf.keras.models.Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.Conv1 = tf.keras.layers.Conv2D(64,(3,3),activation = "relu")
    self.MaxPool = tf.keras.layers.MaxPool2D((2,2))
    self.flatten = tf.keras.layers.Flatten()
    self.Dense1 = tf.keras.layers.Dense(128,activation = "relu")
    self.Output = tf.keras.layers.Dense(10,)#activation ="softmax")
  
  def call(self,x):
    x = self.Conv1(x) 
    x = self.MaxPool(x)
    x = self.flatten(x)
    x = self.Dense1(x)
    x = self.Output(x)
    return x

In [115]:
model = MyModel()

In [116]:
#Define the hyperparameters
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.Accuracy()
optimizer = tf.keras.optimizers.Adam()
batch_size = 32

Selects the metrics to measure the loss and accuracy of the model

In [117]:
#Metrics to measure the loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "train_accuracy")

#test loss and accuracy 
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test-accuracy")

In [118]:
#Using tf.Gradient Tape to Train the model 
@tf.function #Converts a regular python function to a tensorflow graph function 
def train_step(images,labels):
  with tf.GradientTape() as tape:
    #Training = true is there are  layers with different behaviour during training versus inference such as Dropout
    predictions = model(images, training =True)
    #Calculate the loss 
    loss = loss_fn(labels,predictions)
    gradient = tape.gradient(loss,model.trainable_variables) #Get the gradient of the loss function 
    optimizer.apply_gradients(zip(gradient, model.trainable_variables)) #Update the parameters based on the gradient of the loss 

    train_loss(loss)
    train_accuracy(labels,predictions)

Test the model

In [119]:
@tf.function
def test_step(images, labels):
  #Training = false is only needed if there are layers with different behaviours
  #during training versus inference 
  predictions = model(images,training=False)
  t_loss = loss_fn(predictions,labels)  

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [ ]:
EPOCHS = 10 

for epoch in range(EPOCHS):
  #Reset the metrics at the start of the next epoch 
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #for image,labels in train_dataset:
   # train_step(image,labels)
  
  for test_images,test_labels in  test_dataset:
    test_step(test_images,test_labels)
    
  print(f"Epoch:{epoch + 1}/n",
        f"train_loss {train_loss.result()}/n",
        f"Accuracy{train_accuracy.result() * 100}/n",
        f"Test_loss {test_loss.result()}/n",
        f"Test_accuracy {test_accuracy.result()}/n"
        )
  #print(f"train_loss {train_loss.result()}")
  #print(f"")